# Predict Helix Capping Residues #

The goal is to identify residues just before an alpha helix begins or the residues just after the helix ends. This will improve secondary structure predictors becuase they often extend too far or do not start at the right place. 

The CapsDB has annoted sequences of structures of helix capping residues that can be used to train a deep nueral net. We will use a Bidirectional LSTM using phi/psi features to see if it will those will be good predictors.

## 1. Download data ##

## 2. Generate Features ##
### MMTF Pyspark Imports ###

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.webfilters import Pisces
from mmtfPyspark.filters import ContainsLProteinChain
from mmtfPyspark.mappers import StructureToPolymerChains
from mmtfPyspark.ml import ProteinSequenceEncoder
import numpy as np
import pandas as pd
import math
import os

### Custom imports ###

In [3]:
import secondaryStructureExtractorFull
#import mmtfToASA

### Configure Spark Context ###

In [4]:
spark = SparkSession.builder.master("local[8]").appName("DeepCap").getOrCreate()

### Create SQLContext ###

In [5]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import concat, col, lit, array_contains

sqlContext = SQLContext(spark)

### Read in filtered cap+MMTF data from parquet file###

In [15]:
# Read output of above get_dataset operation from parquet file
parquetPath = '/home/ec2-user/SageMaker/ProteinFragmenter/datacaps.parquet'
dataframe = sqlContext.read.parquet(parquetPath)
data = dataframe.toPandas()
data = data.drop('__index_level_0__', axis=1)

capsdb = sqlContext.read.parquet('caps_descriptors.parquet')

### Get Torsion angle and secondary structure info ###

In [16]:
#data.head(10)
#", ".join(data.pdbId.unique())
#temp = data[data.pdbId == '7odc']#.resn.shape
#temp.sort()
#temp

In [17]:
df1 = capsdb.toPandas()
df = pd.merge(data, df1, left_on=('pdbId','chain'), right_on=('pdbid','chain'), how='inner')
df = df[['pdbId', 'chain', 'resi', 'resn', 'phi', 'psi', 'startcap', 'endcap']]


### Create labels

In [18]:
df['is_cap'] = df.apply(lambda x: 1 if (x['resi'] >= x['startcap'] and x['resi'] <= x['endcap']) else 0, axis=1)
df_caps = df.groupby(["pdbId", "chain", "resi"])['is_cap'].max().reset_index()

In [19]:
data_caps = pd.merge(data, df_caps, left_on=('pdbId','chain', 'resi'), right_on=('pdbId','chain', 'resi'), how='inner')

In [20]:
from Bio.PDB.Polypeptide import aa3
one_hot_encoded = pd.DataFrame(data_caps.resn.apply(lambda x: secondaryStructureExtractorFull.get_residue(x)).tolist(), columns=aa3)
one_hot_encoded.head()
data_caps = data_caps.join(one_hot_encoded)
data_caps.head()

,pdbId,chain,resi,resn,phi,psi,is_cap,ALA,CYS,ASP,...,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
0,2ygn,A,1,THR,NaN,163.677383,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2ygn,A,2,GLY,-66.660973,160.703186,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2ygn,A,3,SER,-123.853607,-7.871733,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,2ygn,A,4,LEU,-74.896896,137.483932,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2ygn,A,5,TYR,-134.419830,140.864288,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [27]:
data_caps.head()

,pdbId,chain,resi,resn,phi,psi,is_cap,ALA,CYS,ASP,...,MET,ASN,PRO,GLN,ARG,SER,THR,VAL,TRP,TYR
0,2ygn,A,1,THR,NaN,163.677383,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2ygn,A,2,GLY,-66.660973,160.703186,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2ygn,A,3,SER,-123.853607,-7.871733,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,2ygn,A,4,LEU,-74.896896,137.483932,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2ygn,A,5,TYR,-134.419830,140.864288,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


# Define functions for feature extraction

In [34]:

def is_cap(pdbId, chain, resi, is_cap):
    if is_cap == 1:
        return(1)
    elif is_cap == 0:
        return(0)
    else:
        raise ValueError("is_cap must be 0 or 1")

def angle_to_cos(angle):
    if(angle == 0 or np.isnan(angle)):
        return 0
    else:
        return np.cos(np.pi * angle/180)

def angle_to_sin(angle):
    if(angle == 0 or np.isnan(angle)):
        return 0
    else:
        return np.sin(np.pi * angle/180)
    
def parse_feature_file(filename):
    # Read file
    df = pd.read_csv(filename, sep="\t", skiprows=range(10))
    df = df.reset_index()
    # Set column names
    cnames=[
    "RESIDUE_CLASS1_IS_HYDROPHOBIC",
    "RESIDUE_CLASS1_IS_CHARGED",
    "RESIDUE_CLASS1_IS_POLAR",
    "RESIDUE_CLASS1_IS_UNKNOWN",
    "RESIDUE_CLASS2_IS_NONPOLAR",
    "RESIDUE_CLASS2_IS_POLAR",
    "RESIDUE_CLASS2_IS_BASIC",
    "RESIDUE_CLASS2_IS_ACIDIC",
    "RESIDUE_CLASS2_IS_UNKNOWN",
    ]
    l = [["{}_SHELL{}".format(c, i) for c in cnames] for i in range(6)]
    cnames = [item for sublist in l for item in sublist]
    cnames.insert(0, "env")
    cnames.extend(["hash", "x", "y", "z", "VERBOSITY", "location"])
    df.columns = cnames
    
    df = df.drop(["hash", "x", "y", "z", "VERBOSITY"], axis=1)
    
    # filter out non-AA structures
    aminoAcids = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS',
    'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 
    'TYR', 'VAL']
    df['residue'] = df.location.str[:3]
    df = df[df.residue.isin(aminoAcids)]
    
    # Split identifiers

    # split residue number
    df_residue = df.location.str[3:].str.split(":", 1, expand=True)
    df_residue.columns = ["ordernum", "chainatom"]
    df = pd.merge(df, df_residue, left_index=True, right_index=True)

    # split chain
    df_chain = df.chainatom.str.split("@", 1, expand=True)
    df_chain.columns = ["chain", "atom"]
    df = pd.merge(df, df_chain, left_index=True, right_index=True)
    
    df['pdbId'] = df.env.str[4:8]

    df = df.drop(["env", "location", "chainatom"], axis=1)
    
    #df.ordernum = df.ordernum.astype(int)

    # Aggregate to residue level
    groups = df.groupby(['chain', 'ordernum', 'residue'], sort=False)
    groupnums = groups.ngroup(ascending=True)
    df_agg_max = groups.max().reset_index()
    df_agg_max = df_agg_max.drop(['atom', 'ordernum'], axis=1)
    df_agg_max['resi'] = df_agg_max.index+1
    return(df_agg_max)


# Parse feature files to get additional features

In [ ]:
files = os.listdir("feature/feature-3.1.0/feature_files")
filelist = []
f = open("feature_parsing.log", "w")
for i, filename in enumerate(files):
    print(i, filename)
    f.write("File {}: protein {}\n".format(i, filename))
    df = parse_feature_file("feature/feature-3.1.0/feature_files/{}".format(filename))
    filelist.append(df)

f.close()
feature_df = pd.concat(filelist)

In [56]:
feature_df.to_csv("Feature_vectors.csv")

In [62]:
#data_caps
data_caps2 = data_caps.merge(feature_df, left_on=["pdbId", "chain", "resi"], right_on=["pdbId", "chain", "resi"])

In [96]:
#data_caps2.columns
#data_caps.shape
#feature_df.shape
#data_caps[data_caps.pdbId == "2ygn"]
#feature_df[feature_df.pdbId == "2ygn"]
#data_caps.groupby(['pdbId', 'chain']).ngroup().unique().shape
#feature_df.groupby(['pdbId', 'chain']).ngroup().unique().shape
#data_caps2.groupby(['pdbId', 'chain']).ngroup().unique().shape
train_chains[1].shape

(122, 78)

# Process data into list of arrays

In [89]:
groups = data_caps2.groupby(["pdbId", "chain"])
                           # num pdbs,    max len of seqs, num features

# Check max length of protein chains
# maxlen = 0
# for i, ((pdbid, chain), group) in enumerate(groups):
#     l = 0
#     for j, featuretuple in enumerate(group.itertuples()):
#         l += 1
#         if l > maxlen:
#             maxlen = l
# print(maxlen)

1270


In [92]:
train_chains = []
label_chains = []
laglabel_chains = []

for i, ((pdbid, chain), group) in enumerate(groups):
    # Create empty arrays
    train_chain = np.zeros((1300,78), dtype=float) # max chain length is 1288 residues
    label_chain = np.zeros((1300,1), dtype=int)
    laglabel_chain = np.zeros((5000,1), dtype=int)
    
    # Populate arrays
    for j, featuretuple in enumerate(group.itertuples()):
        train_chain[j, :] = (angle_to_cos(featuretuple.phi), 
                             angle_to_sin(featuretuple.phi), 
                             angle_to_cos(featuretuple.psi), 
                             angle_to_sin(featuretuple.psi),
                            featuretuple.ALA,
                            featuretuple.CYS,
                            featuretuple.ASP,
                            featuretuple.GLU,
                            featuretuple.PHE,
                            featuretuple.GLY,
                            featuretuple.HIS,
                            featuretuple.ILE,
                            featuretuple.LYS,
                            featuretuple.LEU,
                            featuretuple.MET,
                            featuretuple.ASN,
                            featuretuple.PRO,
                            featuretuple.GLN,
                            featuretuple.ARG,
                            featuretuple.SER,
                            featuretuple.THR,
                            featuretuple.VAL,
                            featuretuple.TRP,
                            featuretuple.TYR,
                            featuretuple.RESIDUE_CLASS1_IS_HYDROPHOBIC_SHELL0,
                            featuretuple.RESIDUE_CLASS1_IS_CHARGED_SHELL0,
                            featuretuple.RESIDUE_CLASS1_IS_POLAR_SHELL0,
                            featuretuple.RESIDUE_CLASS1_IS_UNKNOWN_SHELL0,
                            featuretuple.RESIDUE_CLASS2_IS_NONPOLAR_SHELL0,
                            featuretuple.RESIDUE_CLASS2_IS_POLAR_SHELL0,
                            featuretuple.RESIDUE_CLASS2_IS_BASIC_SHELL0,
                            featuretuple.RESIDUE_CLASS2_IS_ACIDIC_SHELL0,
                            featuretuple.RESIDUE_CLASS2_IS_UNKNOWN_SHELL0,
                            featuretuple.RESIDUE_CLASS1_IS_HYDROPHOBIC_SHELL1,
                            featuretuple.RESIDUE_CLASS1_IS_CHARGED_SHELL1,
                            featuretuple.RESIDUE_CLASS1_IS_POLAR_SHELL1,
                            featuretuple.RESIDUE_CLASS1_IS_UNKNOWN_SHELL1,
                            featuretuple.RESIDUE_CLASS2_IS_NONPOLAR_SHELL1,
                            featuretuple.RESIDUE_CLASS2_IS_POLAR_SHELL1,
                            featuretuple.RESIDUE_CLASS2_IS_BASIC_SHELL1,
                            featuretuple.RESIDUE_CLASS2_IS_ACIDIC_SHELL1,
                            featuretuple.RESIDUE_CLASS2_IS_UNKNOWN_SHELL1,
                            featuretuple.RESIDUE_CLASS1_IS_HYDROPHOBIC_SHELL2,
                            featuretuple.RESIDUE_CLASS1_IS_CHARGED_SHELL2,
                            featuretuple.RESIDUE_CLASS1_IS_POLAR_SHELL2,
                            featuretuple.RESIDUE_CLASS1_IS_UNKNOWN_SHELL2,
                            featuretuple.RESIDUE_CLASS2_IS_NONPOLAR_SHELL2,
                            featuretuple.RESIDUE_CLASS2_IS_POLAR_SHELL2,
                            featuretuple.RESIDUE_CLASS2_IS_BASIC_SHELL2,
                            featuretuple.RESIDUE_CLASS2_IS_ACIDIC_SHELL2,
                            featuretuple.RESIDUE_CLASS2_IS_UNKNOWN_SHELL2,
                            featuretuple.RESIDUE_CLASS1_IS_HYDROPHOBIC_SHELL3,
                            featuretuple.RESIDUE_CLASS1_IS_CHARGED_SHELL3,
                            featuretuple.RESIDUE_CLASS1_IS_POLAR_SHELL3,
                            featuretuple.RESIDUE_CLASS1_IS_UNKNOWN_SHELL3,
                            featuretuple.RESIDUE_CLASS2_IS_NONPOLAR_SHELL3,
                            featuretuple.RESIDUE_CLASS2_IS_POLAR_SHELL3,
                            featuretuple.RESIDUE_CLASS2_IS_BASIC_SHELL3,
                            featuretuple.RESIDUE_CLASS2_IS_ACIDIC_SHELL3,
                            featuretuple.RESIDUE_CLASS2_IS_UNKNOWN_SHELL3,
                            featuretuple.RESIDUE_CLASS1_IS_HYDROPHOBIC_SHELL4,
                            featuretuple.RESIDUE_CLASS1_IS_CHARGED_SHELL4,
                            featuretuple.RESIDUE_CLASS1_IS_POLAR_SHELL4,
                            featuretuple.RESIDUE_CLASS1_IS_UNKNOWN_SHELL4,
                            featuretuple.RESIDUE_CLASS2_IS_NONPOLAR_SHELL4,
                            featuretuple.RESIDUE_CLASS2_IS_POLAR_SHELL4,
                            featuretuple.RESIDUE_CLASS2_IS_BASIC_SHELL4,
                            featuretuple.RESIDUE_CLASS2_IS_ACIDIC_SHELL4,
                            featuretuple.RESIDUE_CLASS2_IS_UNKNOWN_SHELL4,
                            featuretuple.RESIDUE_CLASS1_IS_HYDROPHOBIC_SHELL5,
                            featuretuple.RESIDUE_CLASS1_IS_CHARGED_SHELL5,
                            featuretuple.RESIDUE_CLASS1_IS_POLAR_SHELL5,
                            featuretuple.RESIDUE_CLASS1_IS_UNKNOWN_SHELL5,
                            featuretuple.RESIDUE_CLASS2_IS_NONPOLAR_SHELL5,
                            featuretuple.RESIDUE_CLASS2_IS_POLAR_SHELL5,
                            featuretuple.RESIDUE_CLASS2_IS_BASIC_SHELL5,
                            featuretuple.RESIDUE_CLASS2_IS_ACIDIC_SHELL5,
                            featuretuple.RESIDUE_CLASS2_IS_UNKNOWN_SHELL5)
        label_chain[j,0] = is_cap(featuretuple.pdbId, featuretuple.chain, featuretuple.resi, featuretuple.is_cap)
        if (j > 0):
            laglabel_chain[j-1,0] = label_chain[j,0]
    
    # Trim zeros
    trimmed_train = train_chain[~np.all(train_chain == 0, axis=1)]
    trimmed_label = label_chain[:trimmed_train.shape[0]]
    trimmed_laglabel = label_chain[:trimmed_train.shape[0]+1]
    
    # Add chain data to lists of arrays
    train_chains.append(trimmed_train)
    label_chains.append(trimmed_label)
    laglabel_chains.append(trimmed_laglabel)
    

# Write training data to pickle file

In [93]:
import pickle
pickle_out = open("pickled_data/train_chains_78.pickle","wb")
pickle.dump(train_chains, pickle_out)
pickle_out.close()

pickle_out = open("pickled_data/label_chains_78.pickle","wb")
pickle.dump(label_chains, pickle_out)
pickle_out.close()

pickle_out = open("pickled_data/laglabel_chains_78.pickle","wb")
pickle.dump(laglabel_chains, pickle_out)
pickle_out.close()

In [8]:
spark.stop()

# The code below reads in 1-dim (binary) labels and writes back out as 2-dim labels (one-hot)

In [97]:
import pickle
label_chain_in = open("pickled_data/label_chains_78.pickle","rb")
labels = pickle.load(label_chain_in)

newlabels = []
for i, l in enumerate(labels):
    temp = np.zeros([l.shape[0], 2], dtype=int)
    temp[:,1] = l[:,0]
    temp[:,0] = (l[:,0]+1)%2
    newlabels.append(temp)

pickle_out = open("pickled_data/label_chains_78.pickle","wb")
pickle.dump(newlabels, pickle_out)
pickle_out.close()

In [98]:
laglabel_chain_in = open("pickled_data/laglabel_chains_78.pickle","rb")
labelslag = pickle.load(laglabel_chain_in)

newlabelslag = []
for i, l in enumerate(labelslag):
    temp = np.zeros([l.shape[0], 2], dtype=int)
    temp[:,1] = l[:,0]
    temp[:,0] = (l[:,0]+1)%2
    newlabelslag.append(temp)

pickle_out = open("pickled_data/laglabel_chains_78.pickle","wb")
pickle.dump(newlabelslag, pickle_out)
pickle_out.close()

# The code below reads in train/label and writes out lists sorted by chain length

In [31]:
import pickle
train_chain_in = open("pickled_data/train_chains.pickle","rb")
train = pickle.load(train_chain_in)

lens = [len(chain) for chain in train]
inds = range(len(train))
lenSeries = pd.Series(data=lens, index=inds).sort_values()
newInds = lenSeries.index.values
newlist = []
[newlist.append(train[i]) for i in newInds]

pickle_out = open("pickled_data/train_chains_sorted.pickle","wb")
pickle.dump(newlist, pickle_out)
pickle_out.close()

# now sort label list
label_chain_in = open("pickled_data/label_chains.pickle","rb")
labels = pickle.load(label_chain_in)

newlist2 = []
[newlist2.append(labels[i]) for i in newInds]

pickle_out = open("pickled_data/label_chains_sorted.pickle","wb")
pickle.dump(newlist2, pickle_out)
pickle_out.close()

# now sort laglabel list
laglabel_chain_in = open("pickled_data/laglabel_chains.pickle","rb")
labelslag = pickle.load(laglabel_chain_in)

newlist3 = []
[newlist3.append(labelslag[i]) for i in newInds]

pickle_out = open("pickled_data/laglabel_chains_sorted.pickle","wb")
pickle.dump(newlist3, pickle_out)
pickle_out.close()